In [7]:
%matplotlib inline
# -*- coding: utf-8 -*-
"""
Exemple de MNIST avec PyTorch
Exemple avec ResNet6
"""
import torch
torch.manual_seed(0) # Pour résultats reproductibles

# Fonction J d'entropie croisée
import torch.nn.functional as F
fonction_cout = F.cross_entropy

def taux_bonnes_predictions(lot_Y_predictions, lot_Y):
    predictions_categorie = torch.argmax(lot_Y_predictions, dim=1)
    return (predictions_categorie == lot_Y).float().mean()

from torch import nn
# Définition de l'architecture du RNA

class BlocResiduel2d(nn.Module):
    def __init__(self, nb_canaux_in, nb_canaux_out,pas=1):
        super().__init__()
        self.conv1 = nn.Conv2d(nb_canaux_in, nb_canaux_out,kernel_size=3, padding=1, stride=pas)
        self.conv2 = nn.Conv2d(nb_canaux_out, nb_canaux_out,kernel_size=3, padding=1)
        if nb_canaux_in != nb_canaux_out:
            self.conv3 = nn.Conv2d(nb_canaux_in, nb_canaux_out,kernel_size=1, stride=pas)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(nb_canaux_out)
        self.bn2 = nn.BatchNorm2d(nb_canaux_out)
        self.relu = nn.ReLU(inplace=True)

    def forward(self, lot_X):
        lot_Y_predictions = F.relu(self.bn1(self.conv1(lot_X)))
        lot_Y_predictions = self.bn2(self.conv2(lot_Y_predictions))
        if self.conv3:
            lot_X = self.conv3(lot_X)
        lot_Y_predictions += lot_X
        return F.relu(lot_Y_predictions)

modele = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3), # ->(N,64,112,112)
                   nn.BatchNorm2d(64), nn.ReLU(), # ->(N,64,112,112)
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1),# ->(N,64,56,56)
                   BlocResiduel2d(64,64), BlocResiduel2d(64,64) ,# ->(N,64,56,56)
                   nn.AdaptiveMaxPool2d((1,1)), nn.Flatten(), nn.Linear(64,10)
                  )

from torch import optim
optimiseur = optim.SGD(modele.parameters(), lr=0.01)

import torchvision
from torchvision import datasets, models, transforms

pretraitement = transforms.Compose([
transforms.Resize(224),
transforms.CenterCrop(224),
transforms.ToTensor()])

#Chargement des données
ds = torchvision.datasets.MNIST(root = "./data", train = True, download = True, transform = pretraitement)
ds_ent, ds_valid = torch.utils.data.random_split(ds, [50000, 10000])
ds_test = torchvision.datasets.MNIST(root = "./data", train = False, download = True, transform = pretraitement)

#Création du DataLoader avec le dataset
dl_ent = torch.utils.data.DataLoader(ds_ent, batch_size=100, shuffle = True)
dl_valid = torch.utils.data.DataLoader(ds_valid, batch_size=100)

def entrainer(modele, dl_ent, dl_valid, optimiseur, nb_epochs=10):

    # Listes pour les métriques par epoch
    liste_cout_moyen_ent = []
    liste_taux_moyen_ent = []
    liste_cout_moyen_valid = []
    liste_taux_moyen_valid = []
    
    # Boucle d'apprentissage
    for epoch in range(nb_epochs):
        cout_total_ent = 0 # pour cumuler les couts par mini-lot
        taux_bonnes_predictions_ent = 0 # pour cumuler les taux par mini-lot
        modele.train() # Pour certains types de couches (nn.BatchNorm2d, nn.Dropout, ...)
        
        # Boucle d'apprentissage par mini-lot pour une epoch
        for lot_X, lot_Y in dl_ent:
            optimiseur.zero_grad() # Remettre les dérivées à zéro
            lot_Y_predictions = modele(lot_X) # Appel de la méthode forward
            cout = fonction_cout(lot_Y_predictions, lot_Y)
            cout.backward() # Calcul des gradiants par rétropropagation
            with torch.no_grad():
                cout_total_ent +=cout
                taux_bonnes_predictions_ent += taux_bonnes_predictions(lot_Y_predictions, lot_Y)
            optimiseur.step() # Mise à jour des paramètres
            # scheduler.step()
        # Calculer les moyennes par mini-lot
        with torch.no_grad():
            cout_moyen_ent = cout_total_ent/len(dl_ent)
            taux_moyen_ent = taux_bonnes_predictions_ent/len(dl_ent)
       
        modele.eval() # Pour certains types de couches (nn.BatchNorm2d, nn.Dropout, ...)
        with torch.no_grad():
            cout_valid = sum(fonction_cout(modele(lot_valid_X), lot_valid_Y) for lot_valid_X, lot_valid_Y in dl_valid)
            taux_bons_valid = sum(taux_bonnes_predictions(modele(lot_valid_X), lot_valid_Y) for lot_valid_X, lot_valid_Y in dl_valid)
        cout_moyen_valid = cout_valid/len(dl_valid)
        taux_moyen_valid = taux_bons_valid/len(dl_valid)
        print(f'-------- > epoch {epoch+1}:  coût moyen entraînement = {cout_moyen_ent}')
        print(f'-------- > epoch {epoch+1}:  taux moyen entraînement = {taux_moyen_ent}')
        print(f'-------- > epoch {epoch+1}:  coût moyen validation = {cout_moyen_valid}')
        print(f'-------- > epoch {epoch+1}:  taux moyen validation = {taux_moyen_valid}')
    
        liste_cout_moyen_ent.append(cout_moyen_ent)
        liste_taux_moyen_ent.append(taux_moyen_ent)
        liste_cout_moyen_valid.append(cout_moyen_valid)
        liste_taux_moyen_valid.append(taux_moyen_valid)
    
    # Affichage du graphique d'évolution des métriques par epoch
    import numpy as np
    import matplotlib.pyplot as plt
    plt.plot(np.arange(0,nb_epochs),liste_cout_moyen_ent,label='Erreur entraînement')
    plt.plot(np.arange(0,nb_epochs),liste_cout_moyen_valid,label='Erreur validation')
    plt.title("Evolution du coût")
    plt.xlabel('epoch')
    plt.ylabel('moyenne par observation')
    plt.legend(loc='upper center')
    plt.show()
        
    plt.plot(np.arange(0,nb_epochs),liste_taux_moyen_ent,label='Taux bonnes réponses entraînement')
    plt.plot(np.arange(0,nb_epochs),liste_taux_moyen_valid,label='Taux bonnes réponses validation')
    plt.title("Evolution du taux")
    plt.xlabel('epoch')
    plt.ylabel('moyenne par observation')
    plt.legend(loc='center')
    plt.show()

entrainer(modele, dl_ent, dl_valid, optimiseur, nb_epochs=10)

-------- > epoch 1:  coût moyen entraînement = 1.594387173652649
-------- > epoch 1:  taux moyen entraînement = 0.4863798916339874
-------- > epoch 1:  coût moyen validation = 0.4931544065475464
-------- > epoch 1:  taux moyen validation = 0.8498001098632812


KeyboardInterrupt: 